# GLS PERIODOGRAMS - GENERIC SYNTHETIC SAMPLE BATCH FILE PROCESSING - WITH FREQUENCY GRID AT 0.01

In [1]:
# NOTA IMPORTANTE - CÓMO IMPLEMENTAR LO DEL "TIMEOUT".
# https://stackoverflow.com/questions/25027122/break-the-function-after-certain-time
# NO SIRVE - SOLO SIRVE PARA ENTORNOS UNIX

This notebook is a generic notebook encompassing all the previus notebooks for bulk processing.

**Calculation conditions:**

The stars currently under analysis are in the $He_{3}$ instability band, so they are expected to have effective temperatures in the range $T_{eff}\in[3300,\;4300]\;K$, $\log g\in[4.5,\;5.1]$, and masses in the range $M_{star}\in[0.20,\;0.60]\;M_{\odot}$.

According to _Table 2_ in _The theoretical instability strip of M dwarf stars (Rodríguez-López, C., et al. 2014, MNRAS, 438, 2371)_ these stars have typical periods of $20\;min$ to $3\;h$ (corresponding to periods of $P_{pulsation}\in[0.013889,\;0.125000]\;d$, or frequencies of $f_{pulsation}\in[72.0,\;8.0]\;d^{-1}$). Setting a margin over these values, we will set the limits of the periodogram frequencies for periods between $5\;min$ and $10\;h$. In days, this corresponds to a range of $P\in[0.003472,\;0.416667]\;d$ or, equivalently, frequencies in the range $f\in[288.0,\;2.4]\;d^{-1}$.

We will use the _Generalized Lomb Scargle Periodogram_ method, as described in [Zechmeister and Kürster, 2009](https://www.aanda.org/articles/aa/full_html/2009/11/aa11296-08/aa11296-08.html) and implemented by GitHub repository [mzechmeister/GLS](https://github.com/mzechmeister/GLS), using the default _ZK_ normalization.

## Modules and configuration

### Modules

In [2]:
# Modules import:
#from collections import OrderedDict
import pandas as pd
import numpy as np
import time

from IPython.display import clear_output
import warnings

from scipy import stats

# https://github.com/mzechmeister/GLS
from gls import Gls

from astropy.table import Table, QTable
#from astropy.timeseries import TimeSeries
from astropy import units as u
from astropy.io import fits
#from astropy.time import Time

import lightkurve as lk

#%matplotlib inline
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 11, 11

#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})
#sns.set_style("whitegrid")
#sns.set(rc={'figure.figsize':(15,8)})

### Configuration

In [8]:
# USER ENTRY NEEDED --- MODIFY THE CASE SUFFIX HERE:
CASE_SUFFIX = "lS1"

# Configuration:
# Files and folders (WARNING: THIS FOLDER STRUCTURE MUST EXIST PREVIOUSLY):
#GTO_FILE = "../data/RV_ForPG_SyntheticDatasets.csv" # NOTE: initially this should be a copy of the previous file.
if "LP" in CASE_SUFFIX:
    if "T100d" in CASE_SUFFIX:
        GTO_FILE = "../data/RV_ML_subsample_LP_T100d_SyntheticDatasets_with_PG_G01.csv"
    else:
        GTO_FILE = "../data/RV_ML_subsample_LP_SyntheticDatasets_with_PG_G01.csv"
else:
    GTO_FILE = "../data/RV_ML_subsample_SyntheticDatasets_with_PG_G01.csv"
if CASE_SUFFIX[:1] == "l":
    IN_RV_FOLDER = "../data/SYNTH_RV_SAMPLES/" + CASE_SUFFIX[1:] + "_ts_files/"
else:
    IN_RV_FOLDER = "../data/SYNTH_RV_SAMPLES/" + CASE_SUFFIX + "_ts_files/"
if CASE_SUFFIX[:1] == "l":
    INPUT_FILE_COLNAME = CASE_SUFFIX[1:] + '_file'
else:
    INPUT_FILE_COLNAME = CASE_SUFFIX + '_file'

# Remove "G01" substring from some variables:
IN_RV_FOLDER = IN_RV_FOLDER.replace("_G01", "")
INPUT_FILE_COLNAME = INPUT_FILE_COLNAME.replace("_G01", "")

OUT_PROCESSED_FOLDER = "../data/" + CASE_SUFFIX + "_ForPG_RVs_PGs/"
OUT_IMG_FOLDER = "../data/" + CASE_SUFFIX + "_ForPG_RVs_PGs/figures/"
#OUT_GTO_FILE = "../data/GTO_objects_withRVPG.csv"
#OUT_PG_FILE = "../data/GTO_PGs.csv"
#OUT_ERROR_FILE = "../data/GTO_PG_ERRORs.csv"

# Option to generate / save a full fits file:
FITS_FILE = False

# Periodogram constants:
FBEG = 2.4 # d^{-1}, corresponds to a period P=10 hours
FEND = 288.0 # d^{-1}, corresponds to a period P=5 min
# NEW number of frequency points (equal for all the periodograms), according to the new frequency grid of 0.01
# (288.0 - 2.4) / 0.01 = 28560
NUM_POINTS = 28561
#if ('S1' in CASE_SUFFIX) or ('S2' in CASE_SUFFIX):
#    NUM_POINTS = 7201 # 7200 for S1 and S2 time series (calculated from Tobs=4.2d, f_max=288 d^{-1}, n0 = 5)
#else:
#    NUM_POINTS = 3570001 # 3570000 for S3 and S4 time series (choose the same as for CARMENES real stars)


PBEG = None # Default
PEND = None # Default
OFAC = 10 # Default
HIFAC = 1 # Default
#FREQ = np.linspace(start=FBEG, stop=FEND, num=1000001) # 1,000,001 points between 4.8 and 144 d^(-1)
FREQ = np.linspace(start=FBEG, stop=FEND, num=NUM_POINTS)
# Must be compatible with FBEG, FEND values
NORM = "ZK" # Default
if CASE_SUFFIX[:1] == "l":
    LS = True # To calculate the LS (i.e. error column is ignored)
else:
    LS = False # To calculate the LS (i.e. error column is considered)
FAST = False # Default
VERBOSE = False # Default
FAP_LEVELS_PLOT = [0.01, 0.05, 0.10] # FAP reference levels to plot

# PULSATION RANGE OF INTEREST (FOR THE PLOTS):
F_LOW = 8 # P=0.1250 d (3 h)
F_HIGH = 72 # P=0.0139 d (20 min)

# Initial timeout value (to prevent a stuck file to interrupt the whole process)
#INITIAL_TIMEOUT = 600 # Seconds


In [9]:
def rv_load(filename: str):
    '''Load the RV file and returns a lightcurve object'''
    rv_lk = Table.read(filename, format='ascii',
                    names = ['time', 'RV', 'eRV'], units=[u.d, u.meter / u.second, u.meter / u.second])
    rv_lk = lk.LightCurve(time=rv_lk['time'], flux=rv_lk['RV'], flux_err=rv_lk['eRV'])
    return rv_lk

In [10]:
def rv_infer_sampling(rv_lk: lk.LightCurve):
    '''Infer sampling period from light curve'''
    time_diffs = rv_lk['time'][1:] - rv_lk['time'][:-1]
    return np.median(time_diffs)

## Data processing

In [11]:
print("Working with case %s..." %CASE_SUFFIX)
print("LOADING FILE %s..." %GTO_FILE)

Working with case S1...
LOADING FILE ../data/RV_ForPG_SyntheticDatasets.csv...


### Sample data loading

In [12]:
# Load data table:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head(5)

,ID,Pulsating,frequency,amplitudeRV,offsetRV,refepochRV,phase,S1_Ps,S1_Tobs,S2_errorRV_dist_idx,...,S3_Ps_median,S3_Ps_stdev,S3_NumPoints,S4_errorRV_mean,S4_errorRV_median,S4_errorRV_stdev,S1_file,S2_file,S3_file,S4_file
0,Star-00000,True,19.79,0.73,0.0,2.457396e+06,0.02,0.0016,100.0,68,...,9.945182,146.785961,16,2.331250,2.190,0.674906,../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-...,../data/SYNTH_RV_SAMPLES/S2_ForPG_ts_files/S2-...,../data/SYNTH_RV_SAMPLES/S3_ForPG_ts_files/S3-...,../data/SYNTH_RV_SAMPLES/S4_ForPG_ts_files/S4-...
1,Star-00001,False,0.00,0.00,0.0,2.457405e+06,0.00,0.0016,100.0,206,...,17.496205,97.040595,13,1.215385,1.210,0.285189,../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-...,../data/SYNTH_RV_SAMPLES/S2_ForPG_ts_files/S2-...,../data/SYNTH_RV_SAMPLES/S3_ForPG_ts_files/S3-...,../data/SYNTH_RV_SAMPLES/S4_ForPG_ts_files/S4-...
2,Star-00002,False,0.00,0.00,0.0,2.457395e+06,0.00,0.0016,100.0,120,...,10.063827,130.652538,36,1.842500,1.355,0.949773,../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-...,../data/SYNTH_RV_SAMPLES/S2_ForPG_ts_files/S2-...,../data/SYNTH_RV_SAMPLES/S3_ForPG_ts_files/S3-...,../data/SYNTH_RV_SAMPLES/S4_ForPG_ts_files/S4-...
3,Star-00003,True,47.09,0.57,0.0,2.457463e+06,0.74,0.0016,100.0,240,...,13.047931,79.648768,12,2.905833,2.910,0.659425,../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-...,../data/SYNTH_RV_SAMPLES/S2_ForPG_ts_files/S2-...,../data/SYNTH_RV_SAMPLES/S3_ForPG_ts_files/S3-...,../data/SYNTH_RV_SAMPLES/S4_ForPG_ts_files/S4-...
4,Star-00004,True,54.97,0.81,0.0,2.457406e+06,0.32,0.0016,100.0,238,...,3.045991,109.842524,117,1.820085,1.710,0.705275,../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-...,../data/SYNTH_RV_SAMPLES/S2_ForPG_ts_files/S2-...,../data/SYNTH_RV_SAMPLES/S3_ForPG_ts_files/S3-...,../data/SYNTH_RV_SAMPLES/S4_ForPG_ts_files/S4-...


In [13]:
gto.tail(5)

,ID,Pulsating,frequency,amplitudeRV,offsetRV,refepochRV,phase,S1_Ps,S1_Tobs,S2_errorRV_dist_idx,...,S3_Ps_median,S3_Ps_stdev,S3_NumPoints,S4_errorRV_mean,S4_errorRV_median,S4_errorRV_stdev,S1_file,S2_file,S3_file,S4_file
295,Star-00295,True,59.96,0.36,0.0,2.457400e+06,0.06,0.0016,100.0,10,...,8.487002,132.703548,21,2.231905,1.93,0.768056,../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-...,../data/SYNTH_RV_SAMPLES/S2_ForPG_ts_files/S2-...,../data/SYNTH_RV_SAMPLES/S3_ForPG_ts_files/S3-...,../data/SYNTH_RV_SAMPLES/S4_ForPG_ts_files/S4-...
296,Star-00296,True,43.87,0.89,0.0,2.470535e+06,0.20,0.0016,100.0,221,...,10.973284,172.563161,36,2.723889,2.24,1.001891,../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-...,../data/SYNTH_RV_SAMPLES/S2_ForPG_ts_files/S2-...,../data/SYNTH_RV_SAMPLES/S3_ForPG_ts_files/S3-...,../data/SYNTH_RV_SAMPLES/S4_ForPG_ts_files/S4-...
297,Star-00297,True,48.08,1.44,0.0,2.457403e+06,0.60,0.0016,100.0,34,...,21.464115,250.166896,17,1.633529,1.65,0.252958,../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-...,../data/SYNTH_RV_SAMPLES/S2_ForPG_ts_files/S2-...,../data/SYNTH_RV_SAMPLES/S3_ForPG_ts_files/S3-...,../data/SYNTH_RV_SAMPLES/S4_ForPG_ts_files/S4-...
298,Star-00298,True,47.94,1.28,0.0,2.457599e+06,0.48,0.0016,100.0,189,...,8.496524,198.804473,11,13.730000,14.56,4.782614,../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-...,../data/SYNTH_RV_SAMPLES/S2_ForPG_ts_files/S2-...,../data/SYNTH_RV_SAMPLES/S3_ForPG_ts_files/S3-...,../data/SYNTH_RV_SAMPLES/S4_ForPG_ts_files/S4-...
299,Star-00299,True,71.57,1.00,0.0,2.457420e+06,0.03,0.0016,100.0,249,...,5.029960,63.155320,20,1.605000,1.67,0.305900,../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-...,../data/SYNTH_RV_SAMPLES/S2_ForPG_ts_files/S2-...,../data/SYNTH_RV_SAMPLES/S3_ForPG_ts_files/S3-...,../data/SYNTH_RV_SAMPLES/S4_ForPG_ts_files/S4-...


In [14]:
gto.shape

(300, 29)

In [15]:
print(list(gto.columns))

['ID', 'Pulsating', 'frequency', 'amplitudeRV', 'offsetRV', 'refepochRV', 'phase', 'S1_Ps', 'S1_Tobs', 'S2_errorRV_dist_idx', 'S2_errorRV_dist_name', 'S2_errorRV_dist_loc', 'S2_errorRV_dist_scale', 'S2_errorRV_mean', 'S2_errorRV_median', 'S2_errorRV_stdev', 'S3_sampling_idx', 'S3_Tobs', 'S3_Ps_mean', 'S3_Ps_median', 'S3_Ps_stdev', 'S3_NumPoints', 'S4_errorRV_mean', 'S4_errorRV_median', 'S4_errorRV_stdev', 'S1_file', 'S2_file', 'S3_file', 'S4_file']


Generate the proper auxiliary columns (with the basic periodograms results).

In [16]:
if 'n_RV_' + CASE_SUFFIX in gto.columns:
    pass
else:
    gto['n_RV_' + CASE_SUFFIX] = None # Number of points in RV curve
    gto['Ps_RV_' + CASE_SUFFIX] = None # Sampling period (d)
    gto['fs_RV_' + CASE_SUFFIX] = None # Sampling frequency (d^(-1))
    gto['wmean_RV_' + CASE_SUFFIX] = None # Mean of RV
    gto['wrms_RV_' + CASE_SUFFIX] = None # RMS of RV
    gto['info_PG_RV_' + CASE_SUFFIX] = None # Information text about the PG
    gto['maxP_PG_RV_' + CASE_SUFFIX] = None # Max power value in the PG
    gto['maxSNR_PG_RV_' + CASE_SUFFIX] = None # Max power value in the PG
    gto['rms_PG_RV_' + CASE_SUFFIX] = None # RMS value in the PG residuals
    gto['f_PG_RV_' + CASE_SUFFIX] = None # Best frequency in the PG (d^(-1))
    gto['e_f_PG_RV_' + CASE_SUFFIX] = None # Error of the best frequency in the PG (d^(-1))
    gto['Pd_PG_RV_' + CASE_SUFFIX] = None # Best period in the PG (d)
    gto['e_Pd_PG_RV_' + CASE_SUFFIX] = None # Error of the best period in the PG (d)
    gto['Ph_PG_RV_' + CASE_SUFFIX] = None # Best period in the PG (hours)
    gto['e_Ph_PG_RV_' + CASE_SUFFIX] = None # Error of the best period in the PG (hours)
    gto['Pm_PG_RV_' + CASE_SUFFIX] = None # Best period in the PG (minutes)
    gto['e_Pm_PG_RV_' + CASE_SUFFIX] = None # Error of the best period in the PG (minutes)
    gto['A_PG_RV_' + CASE_SUFFIX] = None # Amplitude of the best frequency
    gto['e_A_PG_RV_' + CASE_SUFFIX] = None # Error of the amplitude of the best frequency
    gto['ph_PG_RV_' + CASE_SUFFIX] = None # Amplitude of the best frequency
    gto['e_ph_PG_RV_' + CASE_SUFFIX] = None # Error of the amplitude of the best frequency
    gto['T0_PG_RV_' + CASE_SUFFIX] = None # Reference epoch of the best frequency
    gto['e_T0_PG_RV_' + CASE_SUFFIX] = None # Error of the epoch of the best frequency
    gto['offset_PG_RV_' + CASE_SUFFIX] = None # Offset of the best frequency
    gto['e_offset_PG_RV_' + CASE_SUFFIX] = None # Error of the offset of the best frequency
    gto['FAP_PG_RV_' + CASE_SUFFIX] = None # False alarm probability
    gto['valid_PG_RV_' + CASE_SUFFIX] = None # Flag to indicate if the periodogram calculation succeeded (1) or not (0).
    gto['error_PG_RV_' + CASE_SUFFIX] = None # The error raised during processing. Empty if processing was successful.
    gto['elapsed_time_PG_RV_' + CASE_SUFFIX] = None # The time elapsed in calculation
    gto['fits_file_RV_' + CASE_SUFFIX] = None # The name of the processed fits file.
    gto['PG_file_RV_' + CASE_SUFFIX] = None # The name of the periodogram results file (plain text).
    gto['fig_file_RV_' + CASE_SUFFIX] = None # The name of the figure file.

In [17]:
print(list(gto.columns))

['ID', 'Pulsating', 'frequency', 'amplitudeRV', 'offsetRV', 'refepochRV', 'phase', 'S1_Ps', 'S1_Tobs', 'S2_errorRV_dist_idx', 'S2_errorRV_dist_name', 'S2_errorRV_dist_loc', 'S2_errorRV_dist_scale', 'S2_errorRV_mean', 'S2_errorRV_median', 'S2_errorRV_stdev', 'S3_sampling_idx', 'S3_Tobs', 'S3_Ps_mean', 'S3_Ps_median', 'S3_Ps_stdev', 'S3_NumPoints', 'S4_errorRV_mean', 'S4_errorRV_median', 'S4_errorRV_stdev', 'S1_file', 'S2_file', 'S3_file', 'S4_file', 'n_RV_S1', 'Ps_RV_S1', 'fs_RV_S1', 'wmean_RV_S1', 'wrms_RV_S1', 'info_PG_RV_S1', 'maxP_PG_RV_S1', 'maxSNR_PG_RV_S1', 'rms_PG_RV_S1', 'f_PG_RV_S1', 'e_f_PG_RV_S1', 'Pd_PG_RV_S1', 'e_Pd_PG_RV_S1', 'Ph_PG_RV_S1', 'e_Ph_PG_RV_S1', 'Pm_PG_RV_S1', 'e_Pm_PG_RV_S1', 'A_PG_RV_S1', 'e_A_PG_RV_S1', 'ph_PG_RV_S1', 'e_ph_PG_RV_S1', 'T0_PG_RV_S1', 'e_T0_PG_RV_S1', 'offset_PG_RV_S1', 'e_offset_PG_RV_S1', 'FAP_PG_RV_S1', 'valid_PG_RV_S1', 'error_PG_RV_S1', 'elapsed_time_PG_RV_S1', 'fits_file_RV_S1', 'PG_file_RV_S1', 'fig_file_RV_S1']


### Batch processing of all RV files

In [18]:
n = len(gto)
n

300

#### Batch processing

In [19]:
warnings.filterwarnings('ignore')
# Batch processing:
lapse_list = []
median_lapse = None
# NOTE THE FOR LOOP IS SEPARATED INTO SEVERAL "BATCHES", SO AS TO PREVENT POTENTIAL MEMORY PROBLEMS
# OR TOO MUCH TIME ELAPSED
#for i in range(0, 3): # TEST
for i in range(0, len(gto)): # ALL RECORDS
#for i in range(0, 300):
#for i in range(0, 50):
    clear_output(wait=True)
    start_time = time.time()
    # Names:
    karmn = gto.loc[i, 'ID'] # Synthetic star name
    #commn = gto.loc[i, 'Name'] # Common name
    #tic_id = str(gto.loc[i, 'TIC_id']) # TESS TIC identifier
    print("Record: %d, started at %s"
          %(i, time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime(start_time))))
    if median_lapse is None:
        print("Previous median lapse time: %s" %median_lapse)
    else:
        print("Previous median lapse time: %.2f seconds" %median_lapse)
    print("Processing %s star..." %karmn)
    if True: # TEST
    #try:
        # LOAD RV FILE:
        rv_file = gto.loc[i, INPUT_FILE_COLNAME]
        # NOTE: a modification was needed for synthetic file locations:
        #rv_file = rv_file.replace("./", "../data/")
        print("filename: %s" %rv_file)
        rv_lk = rv_load(rv_file)
        
        # GENERATE PERIODOGRAM:
        gls = Gls((rv_lk['time'].value, rv_lk['flux'].value, rv_lk['flux_err'].value),
              fbeg=FBEG, fend=FEND, Pbeg=PBEG, Pend=PEND, ofac=OFAC, hifac=HIFAC, freq=FREQ,
              norm=NORM, ls=LS, fast=FAST, verbose=VERBOSE)

        # SAVE THE PERIODOGRAM (A PLAIN TEXT FILE):
        pg_filename = OUT_PROCESSED_FOLDER + karmn + "_RV_" + CASE_SUFFIX + "_PG.dat"
        gls.toFile(ofile=pg_filename, header=False)

        # BASIC CALCULATIONS (NEEDED FOR THE TABLE, EVEN IF THE FITS FILE IS NOT SAVED)
        psample = rv_infer_sampling(rv_lk).value
        fsample = 1.0 / psample
        fnyq = 2.0 * fsample
        
        if FITS_FILE == True:
            # GENERATE THE FITS FILE (IF SO INDICATED IN THE OPTIONS):
            # Prepare the fits primary HDU (only header):
            primary_header = fits.Header()
            primary_header['OBJECT'] = (karmn, "KARMENES target name")
            primary_header['NAME'] = (commn, "Object common name")
            primary_header['TIC'] = (tic_id, "Object TESS identifier")
            primary_header['RA_J2000'] = ("00:05:10.89", "Object right ascension (J2000)")
            primary_header['DE_J2000'] = ("+45:47:11.6", "Object declination (J2000)")
            primary_header['SPTYPE'] = ("M1.0 V", "Spectral type")
            primary_header['TEFF_K'] = (3773, "Effective temperature in Kelvin")
            primary_header['LOGG'] = (5.07, "Logarithm of surface gravity")
            primary_header['FEH'] = (-0.04, "Metallicity")
            primary_header['L_LSUN'] = (0.0436229, "Luminosity in Solar luminosities")
            primary_header['R_RSUN'] = (0.48881, "Radius in Solar radii")
            primary_header['M_MSUN'] = (0.4918, "Mass in Solar masses")
            primary_header['D_PC'] = (11.50352803, "Distance in parsec")
            hdu_primary = fits.PrimaryHDU(header=primary_header)

            # Prepare the RV HDU:
            hdu_rv = fits.table_to_hdu(QTable(rv_lk.to_table()))
            hdu_rv.name = "RV_CURVE"
            freq_units = u.d ** (-1)
            hdu_rv.header['OBJECT'] = (karmn, "KARMENES target name")
            hdu_rv.header['PUNIT'] = u.d.to_string(format='fits')
            hdu_rv.header['FUNIT'] = freq_units.to_string(format='fits')
            hdu_rv.header['RVPOINTS'] = (gls.N, "Number of points in the RV curve")
            hdu_rv.header['AVGFLUX'] = (gls._Y, "Average flux of RV curve")
            hdu_rv.header['RMSFLUX'] = (np.sqrt(gls._YY), "Flux RMS of RV curve")
            hdu_rv.header['PSAMPLE'] = (psample, "Inferred cadence in RV curve")
            hdu_rv.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in RV curve")
            hdu_rv.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
                
            # Prepare the PG HDU:
            hdu_pg = fits.table_to_hdu(
                QTable(data=[gls.freq, gls.power], names=['freq', 'power'], 
                       units=[1.0 / u.d, (u.m / u.s) ** 2]))
            hdu_pg.name = "GLS_PG"
            fpoints = len(gls.f)
            fres = (gls.fend - gls.fbeg) / (fpoints - 1)
            hdu_pg.header['OBJECT'] = (karmn, "KARMENES target name")
            hdu_pg.header['FUNIT'] = (freq_units.to_string(format='fits'), "Unit for frequencies")
            hdu_pg.header['PUNIT'] = (u.d.to_string(format='fits'), "Unit for periods")
            hdu_pg.header['PK_FREQ'] = (gls.best['f'], "Frequency of the peak in periodogram")
            hdu_pg.header['PK_POW'] = (gls.pmax, "Power of the peak in periodogram")
            hdu_pg.header['PK_SNR'] = (gls.best['amp'] / gls.rms, "SNR of the peak in periodogram")
            hdu_pg.header['PK_FAP'] = (gls.FAP(Pn=None), "FAP of the peak in periodogram")
            hdu_pg.header['RES_RMS'] = (gls.rms, "RMS of residuals in periodogram")
            hdu_pg.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in RV curve")
            hdu_pg.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
            hdu_pg.header['FPOINTS'] = (fpoints, "Number of points in periodogram")
            hdu_pg.header['FBEG'] = (gls.fbeg, "Start frequency in periodogram")
            hdu_pg.header['FEND'] = (gls.fend, "End frequency in periodogram")
            hdu_pg.header['FRES'] = (fres, "Frequency resolution in periodogram")
            hdu_pg.header['F'] = (gls.best['f'], "Peak best estimate: frequency")
            hdu_pg.header['E_F'] = (gls.best['e_f'], "Peak best estimate: frequency error")
            hdu_pg.header['P'] = (gls.best['P'], "Peak best estimate: period")
            hdu_pg.header['E_P'] = (gls.best['e_P'], "Peak best estimate: period error")
            hdu_pg.header['A'] = (gls.best['amp'], "Peak best estimate: amplitude")
            hdu_pg.header['E_A'] = (gls.best['e_amp'], "Peak best estimate: amplitude error")
            hdu_pg.header['PH'] = (gls.best['ph'], "Peak best estimate: phase")
            hdu_pg.header['E_PH'] = (gls.best['e_ph'], "Peak best estimate: phase error")
            hdu_pg.header['T0'] = (gls.best['T0'], "Peak best estimate: frequency")
            hdu_pg.header['E_T0'] = (gls.best['e_T0'], "Peak best estimate: frequency error")
            hdu_pg.header['OFF'] = (gls.best['offset'], "Peak best estimate: offset")
            hdu_pg.header['E_OFF'] = (gls.best['e_offset'], "Peak best estimate: offset error")
            hdu_pg.header['OFAC'] = (gls.ofac, "Setup: oversampling factor")
            hdu_pg.header['HIFAC'] = (gls.ofac, "Setup: maximum frequency factor")
            hdu_pg.header['NORM'] = (gls.norm, "Setup: normalization type")
            hdu_pg.header['LS'] = (gls.ls, "Setup: conventional Lomb-Scargle calculation")
            hdu_pg.header['FAST'] = (gls.fast, "Setup: fast evaluation, recursive trigonometric")
        
            # Create and save the fits file (if so desired):
            fits_filename = OUT_PROCESSED_FOLDER + karmn + "_RV_" + CASE_SUFFIX + "_PG.fits"
            hdul = fits.HDUList([hdu_primary, hdu_rv, hdu_pg])
            hdul.writeto(fits_filename, overwrite=True)

            # Delete all the HDUs:
            del hdul
            del hdu_primary
            del hdu_rv
            del hdu_pg
        
        else:
            # Do not create the fits file.
            fits_filename = None

        # FILL IN THE DATA IN THE GTO TABLE:
        gto.loc[i, 'n_RV_' + CASE_SUFFIX] = gls.N
        gto.loc[i, 'Ps_RV_' + CASE_SUFFIX] = psample
        gto.loc[i, 'fs_RV_' + CASE_SUFFIX] = fsample
        gto.loc[i, 'wmean_RV_' + CASE_SUFFIX] = gls._Y
        gto.loc[i, 'wrms_RV_' + CASE_SUFFIX] = np.sqrt(gls._YY)
        gto.loc[i, 'info_PG_RV_' + CASE_SUFFIX] = gls.info(stdout=False)
        gto.loc[i, 'maxP_PG_RV_' + CASE_SUFFIX] = gls.power.max()
        gto.loc[i, 'maxSNR_PG_RV_' + CASE_SUFFIX] = gls.best['amp'] / gls.rms
        gto.loc[i, 'rms_PG_RV_' + CASE_SUFFIX] = gls.rms
        gto.loc[i, 'f_PG_RV_' + CASE_SUFFIX] = gls.best['f']
        gto.loc[i, 'e_f_PG_RV_' + CASE_SUFFIX] = gls.best['e_f']
        gto.loc[i, 'Pd_PG_RV_' + CASE_SUFFIX] = gls.best['P']
        gto.loc[i, 'e_Pd_PG_RV_' + CASE_SUFFIX] = gls.best['e_P']
        gto.loc[i, 'Ph_PG_RV_' + CASE_SUFFIX] = 24.0 * gls.best['P']
        gto.loc[i, 'e_Ph_PG_RV_' + CASE_SUFFIX] = 24.0 * gls.best['e_P']
        gto.loc[i, 'Pm_PG_RV_' + CASE_SUFFIX] = 24.0 * 60.0 * gls.best['P']
        gto.loc[i, 'e_Pm_PG_RV_' + CASE_SUFFIX] = 24.0 * 60.0 * gls.best['e_P']
        gto.loc[i, 'A_PG_RV_' + CASE_SUFFIX] = gls.best['amp']
        gto.loc[i, 'e_A_PG_RV_' + CASE_SUFFIX] = gls.best['e_amp']
        gto.loc[i, 'ph_PG_RV_' + CASE_SUFFIX] = gls.best['ph']
        gto.loc[i, 'e_ph_PG_RV_' + CASE_SUFFIX] = gls.best['e_ph']
        gto.loc[i, 'T0_PG_RV_' + CASE_SUFFIX] = gls.best['T0']
        gto.loc[i, 'e_T0_PG_RV_' + CASE_SUFFIX] = gls.best['e_T0']
        gto.loc[i, 'offset_PG_RV_' + CASE_SUFFIX] = gls.best['offset']
        gto.loc[i, 'e_offset_PG_RV_' + CASE_SUFFIX] = gls.best['e_offset']
        gto.loc[i, 'FAP_PG_RV_' + CASE_SUFFIX] = gls.FAP(Pn=None)
        
        # GENERATE THE FIGURE:
        if gto.loc[i, 'Pulsating'] == True:
            star_type = 'Pulsating'
        else:
            star_type = 'Non-pulsating'
        fig = gls.plot(block=False, period=False,
                       fap=FAP_LEVELS_PLOT, gls=True, data=True, residuals=True)
        # Add the reference lines for predicted pulsations:
        fig.axes[0].axvline(F_LOW, color="darkgray", linestyle="--")
        fig.axes[0].axvline(F_HIGH, color="darkgray", linestyle="--")
        figure_title = CASE_SUFFIX + ": %s (%s). Properties: $\\nu$ = %.6f [$d^{-1}$], A = %.6f [$ms^{-1}$]\n" \
            "Detected: P=%.6f [min], f=%.6f [$d^{-1}$], A = %.6f [$ms^{-1}$], FAP=%.6f%%" \
            %(karmn, star_type, gto.loc[i, 'frequency'], gto.loc[i, 'amplitudeRV'], \
              gto.loc[i, 'Pm_PG_RV_' + CASE_SUFFIX], gto.loc[i, 'f_PG_RV_' + CASE_SUFFIX],
              gto.loc[i, 'A_PG_RV_' + CASE_SUFFIX], 100.0 * gto.loc[i, 'FAP_PG_RV_' + CASE_SUFFIX])
        fig.suptitle(figure_title, fontdict = {'fontsize' : 36})
        fig.tight_layout()
        # Save the figure to disk:
        fig_file = OUT_IMG_FOLDER + karmn + "_RV_" + CASE_SUFFIX + "_PG.jpg"
        fig.savefig(fig_file)
        plt.close() # Prevent the figure from showing.

        # SET THE RECORD CALCULATION AS VALID AND STORE THE REULTING FILENAMES
        gto.loc[i, 'valid_PG_RV_' + CASE_SUFFIX] = 1
        gto.loc[i, 'error_PG_RV_' + CASE_SUFFIX] = ""
        gto.loc[i, 'fits_file_RV_' + CASE_SUFFIX] = fits_filename
        gto.loc[i, 'PG_file_RV_' + CASE_SUFFIX] = pg_filename
        gto.loc[i, 'fig_file_RV_' + CASE_SUFFIX] = fig_file

        # UPDATE THE AVERAGE RECORD PROCESSING TIME:
        lapse = time.time() - start_time
        lapse_list.append(lapse)
        median_lapse = np.nanmedian(lapse_list)
        gto.loc[i, 'elapsed_time_PG_RV_' + CASE_SUFFIX] = lapse
        
        # SAVE THE UPDATED GTO TABLE TO DISK:
        gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        
        # Report successful execution:
        print("Elapsed time: %.2f seconds" %lapse)
        print("... SUCCESS.")
        
        # Clear memory (delete the 'gls' object'):
        try:
            del gls
        except:
            pass
        
    else: # TEST
    #except Exception as e:
        # Some error happened, establish the record as not valid and record the error:
        error = "*** Some ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, e)
        print(error)
        gto.loc[i, 'valid_PG_RV_' + CASE_SUFFIX] = 0
        gto.loc[i, 'error_PG_RV_' + CASE_SUFFIX] = e
        
        # Try to update the record, and save the file:
        try:
            # UPDATE THE AVERAGE RECORD PROCESSING TIME:
            lapse = time.time() - start_time
            lapse_list.append(lapse)
            median_lapse = np.nanmedian(lapse_list)
            gto.loc[i, 'elapsed_time_PG_RV_' + CASE_SUFFIX] = lapse
            print("Elapsed time: %.2f seconds" %lapse)

            # SAVE THE UPDATED GTO TABLE TO DISK:
            gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        except Exception as e2:
            error_2 = "*** Additional ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, str(e2))
            print(error)
            gto.loc[i, 'error_PG_RV_' + CASE_SUFFIX] = gto.loc[i, 'error_PG_RV_' + CASE_SUFFIX] + "/" + str(e2)

        # Clear memory (delete the 'gls' object'):
        try:
            del gls
        except:
            pass
print("\n*** COMPLETED ***")

Record: 196, started at 13/01/2023, 07:38:20
Previous median lapse time: 229.93 seconds
Processing Star-00196 star...
filename: ../data/SYNTH_RV_SAMPLES/S1_ForPG_ts_files/S1-RV_LP_T100d_Star-00196.dat


KeyboardInterrupt: 

## Check the calculations that took longer to complete

We now check the calculations that took longer, so as to try to repeat them (as it could be due to a problem with the computer itself - for example, the program stopped when the computer went into "sleep" mode).

In [14]:
gto[['ID', 'elapsed_time_PG_RV_' + CASE_SUFFIX]].sort_values(by='elapsed_time_PG_RV_' + CASE_SUFFIX, ascending=False) \
    .head(10)

,ID,elapsed_time_PG_RV_S2_LP_T100d_G01
49,Star-00049,153.657579
46,Star-00046,153.5002
48,Star-00048,151.744939
47,Star-00047,146.551535
45,Star-00045,144.211311
43,Star-00043,135.262579
44,Star-00044,135.202618
42,Star-00042,129.288054
41,Star-00041,128.693189
40,Star-00040,126.559512


## Summary of calculated periodograms

Number of objects with RV PG properly calculated:

In [15]:
gto[gto['valid_PG_RV_' + CASE_SUFFIX] == 1.0]

,ID,Pulsating,frequency,amplitudeRV,offsetRV,refepochRV,phase,S1_Ps,S1_Tobs,S2_errorRV_dist_idx,...,e_T0_PG_RV_S2_LP_T100d_G01,offset_PG_RV_S2_LP_T100d_G01,e_offset_PG_RV_S2_LP_T100d_G01,FAP_PG_RV_S2_LP_T100d_G01,valid_PG_RV_S2_LP_T100d_G01,error_PG_RV_S2_LP_T100d_G01,elapsed_time_PG_RV_S2_LP_T100d_G01,fits_file_RV_S2_LP_T100d_G01,PG_file_RV_S2_LP_T100d_G01,fig_file_RV_S2_LP_T100d_G01
0,Star-00000,True,10.33,1.14,0.0,2.457582e+06,0.90,0.0016,100.0,209,...,0.000105,-0.006924,0.005497,0.0,1,,85.770306,None,../data/S2_LP_T100d_G01_RVs_PGs/Star-00000_RV_...,../data/S2_LP_T100d_G01_RVs_PGs/figures/Star-0...
1,Star-00001,True,14.92,1.30,0.0,2.457522e+06,0.02,0.0016,100.0,66,...,0.000067,-0.001321,0.005802,0.0,1,,82.85591,None,../data/S2_LP_T100d_G01_RVs_PGs/Star-00001_RV_...,../data/S2_LP_T100d_G01_RVs_PGs/figures/Star-0...
2,Star-00002,False,0.00,0.00,0.0,2.457549e+06,0.00,0.0016,100.0,1,...,0.000136,0.000589,0.005988,0.099765,1,,83.056173,None,../data/S2_LP_T100d_G01_RVs_PGs/Star-00002_RV_...,../data/S2_LP_T100d_G01_RVs_PGs/figures/Star-0...
3,Star-00003,True,28.74,0.90,0.0,2.457460e+06,0.29,0.0016,100.0,189,...,0.000044,-0.005258,0.00502,0.0,1,,82.320757,None,../data/S2_LP_T100d_G01_RVs_PGs/Star-00003_RV_...,../data/S2_LP_T100d_G01_RVs_PGs/figures/Star-0...
4,Star-00004,True,60.83,1.44,0.0,2.457451e+06,0.10,0.0016,100.0,225,...,0.000014,-0.001591,0.005326,0.0,1,,82.564754,None,../data/S2_LP_T100d_G01_RVs_PGs/Star-00004_RV_...,../data/S2_LP_T100d_G01_RVs_PGs/figures/Star-0...
5,Star-00005,True,17.00,0.13,0.0,2.457419e+06,0.11,0.0016,100.0,191,...,0.000619,0.003263,0.005395,0.0,1,,82.61727,None,../data/S2_LP_T100d_G01_RVs_PGs/Star-00005_RV_...,../data/S2_LP_T100d_G01_RVs_PGs/figures/Star-0...
6,Star-00006,True,38.25,0.80,0.0,2.457545e+06,0.44,0.0016,100.0,19,...,0.000045,0.004681,0.006128,0.0,1,,83.227316,None,../data/S2_LP_T100d_G01_RVs_PGs/Star-00006_RV_...,../data/S2_LP_T100d_G01_RVs_PGs/figures/Star-0...
7,Star-00007,True,68.59,0.16,0.0,2.457574e+06,1.00,0.0016,100.0,208,...,0.000145,0.000562,0.007442,0.0,1,,83.370455,None,../data/S2_LP_T100d_G01_RVs_PGs/Star-00007_RV_...,../data/S2_LP_T100d_G01_RVs_PGs/figures/Star-0...
8,Star-00008,True,46.16,0.42,0.0,2.457433e+06,0.63,0.0016,100.0,120,...,0.00006,-0.001793,0.005239,0.0,1,,84.713263,None,../data/S2_LP_T100d_G01_RVs_PGs/Star-00008_RV_...,../data/S2_LP_T100d_G01_RVs_PGs/figures/Star-0...
9,Star-00009,True,12.83,1.25,0.0,2.458828e+06,0.83,0.0016,100.0,178,...,0.000108,-0.002673,0.007538,0.0,1,,84.009919,None,../data/S2_LP_T100d_G01_RVs_PGs/Star-00009_RV_...,../data/S2_LP_T100d_G01_RVs_PGs/figures/Star-0...


In [16]:
gto[gto['valid_PG_RV_' + CASE_SUFFIX] == 0.0][['ID', INPUT_FILE_COLNAME, 'valid_PG_RV_' + CASE_SUFFIX, 'error_PG_RV_' + CASE_SUFFIX]]

,ID,S2_LP_T100d_file,valid_PG_RV_S2_LP_T100d_G01,error_PG_RV_S2_LP_T100d_G01


All periodograms were calculated correctly for all the objects in the sample.

# Summary

**OBSERVATIONS AND CONCLUSIONS:**
- We have completed the basic LS / GLS periodogram calculation for the radial velocity (RV) curves of the generic synthetic sample, and stored the results.
- All 1000 objects were calculated correctly.